### Data Reading

In [16]:
import pandas as pd

# Function to load Quran data from CSV
def load_quran_data(file_path):
    # Load CSV and ensure the relevant columns are read properly
    return pd.read_csv(file_path)

# Function to get the specific Ayat range from a Surah

def get_ayat_range(data, surah_index, start_ayah, end_ayah):
    # Filter based on the given Surah index and Ayah range
    filtered_data = data[(data['Sura Index'] == surah_index) & 
                        (data['Aya Index'] >= start_ayah) & 
                        (data['Aya Index'] <= end_ayah)]
    return filtered_data



## Text Segmentation Function

In [17]:
# Function to concatenate Ayat with underscores and segment into groups
qurani_symbol=['ۚ','ۗ','ۖ','ۛ','ۜ']
shaddah = '\u0651'
sukoon = '\u0652'
arabic_diacritics = [
    '\u0650',  # Arabic Kasra (Zeer)
    '\u064E',  # Arabic Fatha (Zabr)
    '\u064F',  # Arabic Damma (Paish)
    '\u064B',  # Double Zabar (Tanween Fath)
    '\u064C',  # Double Paish (Tanween Damm)
    '\u064D',  # Double Zeer (Tanween Kasr)
    '\u0653',  # Ulta Paish (Maddah)
    '\u0670',  # Khari Zabar (Alef Superscript or Hamzatul Wasl)
    '\u0654',  # Khari Paish (Hamza on Waw)
    '\u0655',  # Khari Zeer (Hamza on Ya)
]

def concatenate_and_segment(ayat_data, words_per_segment, symbols,non_stopping_list,mendatory_stopping,surah,start_ayah,end_ayah):
    # Concatenate all Ayat text into a single string separated by underscores
    full_text = '۝ '.join(ayat_data['Aya Text'].tolist())
    # Split the full text into words
    words = full_text.split()
    segments = []
    segment = []
    not_ending_word=['ﹾ','ﹼ']
    # Loop through each ayah in ayat_data['Aya Text']
    result_dict = {}
    for index, i in enumerate(ayat_data['Aya Text']):
        # Assuming canSegment takes surah, start_ayah, and length of i as arguments
        check = canSegment(surah, start_ayah + index, len(i))
        # Store the result in the dictionary with index as the key
        result_dict[index] = check
    # Optional: print the dictionary to see the results
    print(result_dict)
    i = 0
    j = 0
    while i < len(words):
        if (not result_dict[j]):
            segment.append(words[i])
            if(words[i][-1] in symbols):
                segments.append(' '.join(segment))
                segment = []
                j+=1
            i+=1
            continue
        if i != len(words)-1 and words[i+1] in qurani_symbol:
            words[i]=words[i][:-1]+words[i+1]
            del words[i+1]
        if i != len(words)-1 and words[i+1] in non_stopping_list:
            words[i]=words[i]+non_stopping_list[0]
            del words[i+1]
        if i != len(words)-1 and words[i+1] in mendatory_stopping:
            if words[i+1]=="۩":
                words[i]=words[i]+'۝'
            else:
                words[i]=words[i]+mendatory_stopping[0]
            del words[i+1]
        if(words[i]=="۞"):
            del words[i]
        else:
            segment.append(words[i])
        # Check if we reached the specified number of words per segment
        flage=True
        try:
            for index in range(len(words[i+1])):
                # Check the first two characters of the word
                if words[i+1][index] in arabic_diacritics:
                    flage=True
                    break
                elif words[i+1][index] in shaddah or words[i+1][index] in sukoon:
                    flage=False
                    break
        except IndexError as e:
            print(f"IndexError encountered: {e}. Please check the length of 'last_word' or 'text_list'.")
        try:
            if (len(segment) >= words_per_segment and words[i][-1] not in non_stopping_list and flage):
                #and \
            # ((words[i][0] not in not_ending_word and words[i][1] not in not_ending_word and words[i][2] not in not_ending_word and words[i][3] not in not_ending_word and words[i][4] not in not_ending_word and words[i][6] not in not_ending_word and words[i][5] not in not_ending_word) and \
            # (words[i+1][0] not in not_ending_word and words[i+1][1] not in not_ending_word and words[i+1][2] not in not_ending_word and words[i+1][3] not in not_ending_word and words[i+1][4] not in not_ending_word and words[i+1][5] not in not_ending_word and words[i+1][6] not in not_ending_word)):
                #print (words[i],"--->",words[i][0],words[i][1],words[i][2],words[i][3],words[i][4])
                segments.append(' '.join(segment))
                segment = []  # Reset the segment
            # Check for the symbol in the next word
            if (words[i][-1] in symbols or words[i][-1] in mendatory_stopping):
                # if (not ((words[i][0] not in not_ending_word and words[i][1] not in not_ending_word and words[i][2] not in not_ending_word and words[i][3] not in not_ending_word) and \
                # (words[i+1][0] not in not_ending_word and words[i+1][1] not in not_ending_word and words[i+1][2] not in not_ending_word and words[i+1][3] not in not_ending_word))) or \
                # words[i][-1] in mendatory_stopping:
                if words[i][-1] in symbols:
                    j+=1
                segments.append(' '.join(segment))
                segment = []  # Reset the segment
                
        except IndexError as e:
                print(f"IndexError encountered: {e}. Please check the length of 'last_word' or 'text_list'.")
        except Exception as e:
                print(f"An error occurred: {e}")
        i+=1
    
    # If there are remaining words, add them to the segments
    if segment:
        segments.append(' '.join(segment))
    
    return segments


In [ ]:
import json

# Load the Quran data
quran_data = load_quran_data('./quran_updated.csv')
ayat_data = get_ayat_range(quran_data, 1, 1, 7)
# Load data from output.json222
with open('./output.json', 'r') as file:
    data = json.load(file)

# Filter ayah and surah where deletions > 0
result = []
for entry in data:
    ayah = entry.get('ayah')
    surah = entry.get('surah')
    deletions = entry.get('stats', {}).get('deletions', 0)
    segments = entry.get('segments')
    
    if deletions >= 1:
        result.append({
            'surah': surah,
            'ayah': ayah,
            'deletions': deletions,
            'segments': segments
        })

# Display the result
i=0
for r in result:
    # print(f"Surah: {r['surah']}, Ayah: {r['ayah']}, Deletions: {r['deletions']}")
    #print(ayat_data["surah"]['ayah'])
    surah_name = ayat_data.loc[ayat_data['Sura Index'] == r['surah'], 'Sura Name'].values
    aya_index = ayat_data.loc[ayat_data['Sura Index'] == r['surah'], 'Aya Index'].values
    i+=1

# Optionally, save the filtered result to a new JSON file
# with open('filtered_ayahs.json', 'w') as outfile:
#     json.dump(result, outfile, indent=4)

def canSegment(surah, ayah, lengthWord):
    for r in result:
        if r['surah'] == surah and r['ayah'] == ayah and len(r['segments']) != lengthWord:
            print(f"Surah: {r['surah']}, Ayah: {r['ayah']}, Deletions: {r['deletions']}, Segment Length: {r['segments']}")
            return False
    return True
    


In [ ]:
def process_quran_ayat(file_path, surah_index, start_ayah, end_ayah, words_per_segment):
    # Load Quran data from CSV
    quran_data = load_quran_data(file_path)
    
    # Get the Ayat within the specified Surah and range
    ayat_data = get_ayat_range(quran_data, surah_index, start_ayah, end_ayah)
    
    if ayat_data.empty:
        print("No Ayat found for the given Surah and Ayat range.")
        return
    
    # Concatenate and segment the Ayat data
    segments = concatenate_and_segment(ayat_data, words_per_segment,stopping_list,non_stopping_list,mendatory_stopping,surah_index,start_ayah,end_ayah)
    
    # Display the segmented words
    for segment in segments:
        print(segment)
    return segments
stopping_list=[
    "۝"]
mendatory_stopping = ["\u06D8","۞","۩" ]
non_stopping_list=["\u06D9"]
# Example usage
if __name__ == "__main__":
    # Quran CSV file path
    file_path = 'quran.csv'
    text_list=process_quran_ayat(file_path, 75, 1,27,1)

Surah: 75, Ayah: 5, Deletions: 1, Segment Length: [[0, 1, 2700, 2780], [1, 2, 2790, 4860], [2, 3, 4870, 7240], [3, 4, 7250, 9090], [4, 5, 9100, 12140]]
Surah: 75, Ayah: 11, Deletions: 1, Segment Length: [[0, 1, 4050, 6480], [1, 2, 6490, 7550]]
{0: True, 1: True, 2: True, 3: True, 4: False, 5: True, 6: True, 7: True, 8: True, 9: True, 10: False, 11: True, 12: True, 13: True, 14: True, 15: True, 16: True, 17: True, 18: True, 19: True, 20: True, 21: True, 22: True, 23: True, 24: True, 25: True, 26: True}
IndexError encountered: list index out of range. Please check the length of 'last_word' or 'text_list'.
لَا
أُقْسِمُ
بِيَوْمِ الْقِيَامَةِ۝

وَلَا
أُقْسِمُ
بِالنَّفْسِ اللَّوَّامَةِ۝

أَيَحْسَبُ الْإِنسَانُ
أَلَّن نَّجْمَعَ
عِظَامَهُ۝

بَلَىٰ
قَادِرِينَ
عَلَىٰ
أَن نُّسَوِّيَ
بَنَانَهُ۝

بَلْ يُرِيدُ الْإِنسَانُ لِيَفْجُرَ أَمَامَهُ۝
يَسْأَلُ
أَيَّانَ
يَوْمُ الْقِيَامَةِ۝

فَإِذَا
بَرِقَ الْبَصَرُ۝

وَخَسَفَ الْقَمَرُ۝

وَجُمِعَ الشَّمْسُ
وَالْقَمَرُ۝

يَقُولُ الْإِنسَانُ
يَوْمَئِذٍ
أَيْنَ

In [20]:
!pip install flask


In [21]:
def update_segments_list(surah, start_ayah, end_ayah, segments):
    result = []
    current_segment_index = 0

    # Loop through each ayah from start to end
    for ayah_number in range(start_ayah, end_ayah + 1):
        ayah_segments = []
        
        # Collect segments until we reach the next end of ayah marker
        while current_segment_index < len(segments):
            segment = segments[current_segment_index]
            ayah_segments.append({
                'id': current_segment_index,  # Assigning the segment index as the ID
                'segment': segment
            })
            current_segment_index += 1
            
            # Check if this segment is an end of ayah marker
            if segment.endswith('۝'):
                break  # Stop collecting segments for this ayah

        # Append the segments for the current ayah to the result|
        result.append({
            'surah': surah,
            'ayah': ayah_number,
            'segments': ayah_segments
        })

        # Stop if we've reached the end of segments
        if current_segment_index >= len(segments):
            break

    return result


In [22]:
import pandas as pd
import os

def segment_quran(ayat_data, words_per_segment=5):
    """
    Function to segment Ayat text into chunks of 'words_per_segment' words and keep track of corresponding audio files.
    
    Args:
    ayat_data (pd.DataFrame): DataFrame containing Sura Index, Aya Index, Aya Text, and MP3 File Path.
    words_per_segment (int): Number of words per segment (default is 5).
    
    Returns:
    segmented_text_audio (list): List of dictionaries containing segmented text and corresponding audio file paths.
    """
    
    segmented_text_audio = []
    
    # Iterate over each row in the ayat_data DataFrame
    for index, row in ayat_data.iterrows():
        sura_index = row['Sura Index']
        aya_index = row['Aya Index']
        aya_text = row['Aya Text']
        mp3_file = row['MP3 File Path']
        
        # Split the Ayah text into words
        words = aya_text.split()
        
        # Segment into chunks of 'words_per_segment'
        segments = [words[i:i+words_per_segment] for i in range(0, len(words), words_per_segment)]
        
        # Iterate over each segment and create a corresponding dictionary
        for segment in segments:
            segmented_text_audio.append({
                'Sura Index': sura_index,
                'Aya Index': aya_index,
                'Segmented Text': ' '.join(segment),
                'Audio File Path': mp3_file  # Same audio file for all segments (segmenting audio requires further processing)
            })
    
    return segmented_text_audio

# Load your CSV file into a DataFrame
file_path = "C:/WorkingFolder/FYP/Ayat-segmentation/quran_updated.csv"
ayat_data = pd.read_csv(file_path)

# Segment the text and match it with the corresponding audio
segmented_data = segment_quran(ayat_data, words_per_segment=5)

# Convert the list of dictionaries into a DataFrame
segmented_df = pd.DataFrame(segmented_data)

# Save the segmented data to a new CSV file
output_file = "C:/WorkingFolder/FYP/Ayat-segmentation/segmented_quran.csv"
segmented_df.to_csv(output_file, index=False)

print("Segmentation complete and saved to:", output_file)


Segmentation complete and saved to: C:/WorkingFolder/FYP/Ayat-segmentation/segmented_quran.csv


In [24]:
canSegment(1,1,6)

Surah: 1, Ayah: 1, Deletions: 1, Segment Length: [[0, 1, 0, 960], [1, 2, 970, 1420], [2, 3, 1430, 2670], [3, 4, 2680, 6210]]


False

### Ayat Text and Voice Segmentation API

In [26]:
import os
import torch
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# Define path to your custom fine-tuned model
model_path = "./whisper-small/whisper-small"  # Replace with your model path
print(model_path)
# Load model and processor with mismatched size handling
model = WhisperForConditionalGeneration.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)



./whisper-small/whisper-small


In [27]:
import os
import torch
from datasets import load_dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# Define path to your custom fine-tuned model
MODEL_NAME = "openai/whisper-small"  # Replace with the desired Whisper model size
processor2 = WhisperProcessor.from_pretrained(MODEL_NAME, language="ar", task="transcribe")
model2 = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)



In [ ]:
import os
import torch
import librosa
import re
import json
from difflib import SequenceMatcher
from flask import Flask, request, jsonify
from transformers import WhisperProcessor, WhisperForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

def remove_diacritics(text):
    """Removes Arabic diacritics (harakat) from the text."""
    arabic_diacritics = re.compile(r"[\u064B-\u0652]")  # Range of diacritics
    return arabic_diacritics.sub("", text)

def get_best_match(word, words_list):
    """Finds the best matching word from the given list using SequenceMatcher."""
    best_match = None
    highest_ratio = 0
    
    for w in words_list:
        similarity = SequenceMatcher(None, word, w).ratio()
        if similarity > highest_ratio:
            highest_ratio = similarity
            best_match = w
    
    return best_match, highest_ratio

def calculate_error_rate(correct_text, transcription1, transcription2):
    print(correct_text+"--------------"+transcription1+"--------------"+transcription2)
    correct_words = correct_text.split()
    transcribed_words_1 = transcription1.split()
    transcribed_words_2 = remove_diacritics(transcription2).split()
    print(correct_words, transcribed_words_1, transcribed_words_2)
    all_transcribed_words = set(transcribed_words_1 + transcribed_words_2)  # Combine both transcriptions
    wrong_words = []
    incorrect_count = 0
    
    for correct_word in correct_words:
        # Check if the correct word exists in either transcription
        if correct_word in all_transcribed_words:
            continue  

        # If not found directly, check for similar words
        best_match, similarity = get_best_match(correct_word, all_transcribed_words)
        
        if best_match and similarity >= 0.7:  # Accept minor variations
            continue  

        # If no close match is found, mark as incorrect
        wrong_words.append(correct_word)
        incorrect_count += 1

    total_words = len(correct_words)
    error_rate = incorrect_count / total_words if total_words > 0 else 0

    return {
        "error_rate": round(error_rate, 2),
        "wrong_words": wrong_words
    }


def transcribe_audio(audio_path):
    
# 🔹 Ensure correct padding token
    processor.tokenizer.pad_token = processor.tokenizer.eos_token  # Fix padding issue
    audio, sampling_rate = librosa.load(audio_path, sr=16000)
    # 🔹 Prepare input
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="ar", task="transcribe")
    inputs = processor(
        audio,
        sampling_rate=sampling_rate,
        return_tensors="pt",
        padding=True,  # Ensures correct padding
        language="ar"  # Helps with correct Arabic transcription
    )

    input_features = inputs.input_features.to(device)

    # 🔹 Generate transcription
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            max_length=256,  # Reduce to avoid extra words
            num_beams=1,  # Improves accuracy
            length_penalty=0.8,  # Avoid overgeneration0
            forced_decoder_ids=forced_decoder_ids,
        )

    # 🔹 Decode and post-process output
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()

    # 🔹 Remove extra words using punctuation (adjust as needed)
    transcription = transcription.split("ۚ ")[0]  # Remove extra words after comma

    return transcription

# Flask API route




In [ ]:
import json
import pandas as pd
import os
from pydub import AudioSegment
from pydub.utils import which
from flask import Flask, request, jsonify, send_file
from io import BytesIO
from zipfile import ZipFile

from flask import Flask, request, jsonify
app = Flask(__name__)

# Directory to store segmented audio files
AUDIO_OUTPUT_DIR = 'segmented_audio'
if not os.path.exists(AUDIO_OUTPUT_DIR):
    os.makedirs(AUDIO_OUTPUT_DIR)

# Load output.json data
with open('output.json', 'r', encoding='utf-8') as f:
    output_data = json.load(f)

# Load the Quran audio details
quran_audio_df = pd.read_csv('quran_updated.csv')

def segment_audio_function(surah_index, ayah_index, input_segments):
    """
    Function to process the audio segmentation.

    Args:
    surah_index (int): The surah number.
    ayah_index (int): The ayah number.
    input_segments (list): List of segments with 'id' and 'segment' fields.

    Returns:
    list: A list of dictionaries containing segmented audio information or error.
    """
    AUDIO_OUTPUT_DIR = 'segmented_audio'
    if not os.path.exists(AUDIO_OUTPUT_DIR):
        os.makedirs(AUDIO_OUTPUT_DIR)
    else:
        for file in os.listdir(AUDIO_OUTPUT_DIR):
            os.remove(os.path.join(AUDIO_OUTPUT_DIR, file))
    ayah_data = next((item for item in output_data if item['surah'] == surah_index and item['ayah'] == ayah_index), None)
    
    if ayah_data is None:
        return {"error": "Ayah not found"}

    # Prepare the result segments
    segments_word_length = []
    for segment in input_segments:
        segments_word_length.append(len(segment["segment"].split()))

    segment_audio_time = []
    x = 0
    i = 0
    total_word_length = 0
    per_segment_time = [0]
    
    for seg in ayah_data['segments']:
        if x == 0:
            total_word_length = segments_word_length[i]
        end_time = seg[3]
        starting_index = seg[0]
        ending_index = seg[1]
        per_segment_time.append(end_time)
        x = x + (ending_index - starting_index)
        
        if x == total_word_length:
            x = 0
            i += 1
            segment_audio_time.append(per_segment_time)
            per_segment_time = [end_time]

    if per_segment_time != [] and segment_audio_time == []:
        segment_audio_time.append(per_segment_time)

    # Fetch the corresponding audio file path from the CSV
    audio_path = quran_audio_df[
        (quran_audio_df['Sura Index'] == surah_index) &
        (quran_audio_df['Aya Index'] == ayah_index)
    ]['MP3 File Path'].values

    if not audio_path:
        return {"error": "Audio file not found"}
    
    full_audio_path = audio_path[0]
    full_audio_path = os.path.abspath(full_audio_path)

    try:
        full_audio = AudioSegment.from_mp3(full_audio_path)
    except FileNotFoundError:
        return {"error": f"Audio file not found at: {full_audio_path}"}

    audio_segment_entry = {
        'segment_audio_time': segment_audio_time,
        'audio_path': full_audio_path
    }
    return audio_segment_entry

def save_segmented_audio(segment_audio_time, full_audio_path):
    """
    Save the segmented audio files based on segment times.

    Args:
    segment_audio_time (list): List of segments with start and end times.
    full_audio_path (str): The path of the full audio file.

    Returns:
    list: List of paths to the saved audio segments.
    """
    saved_segments = []
    
    full_audio = AudioSegment.from_mp3(full_audio_path)
    print(full_audio_path)


    for i, time_segment in enumerate(segment_audio_time):
        start_time = time_segment[0]
        end_time = time_segment[-1]
        audio_segment = full_audio[start_time:end_time]
        output_file_name = f"segment_{i + 1}.mp3"
        output_path = os.path.join(AUDIO_OUTPUT_DIR, output_file_name)
        audio_segment.export(output_path, format="mp3")
        saved_segments.append(output_path)

    return saved_segments



@app.route('/segment-audio', methods=['POST'])
def segment_audio():
    """
    API Endpoint to segment audio based on Surah and Ayah details.
    """
    data = request.json

    surah_index = data.get('surah')
    ayah_index = data.get('ayah')
    input_segments = data.get('segments')

    if not surah_index or not ayah_index or not input_segments:
        return jsonify({"error": "Invalid input data"}), 400

    # Call the function to segment audio
    result = segment_audio_function(surah_index, ayah_index, input_segments)

    if 'segment_audio_time' in result:
        # Save the segmented audio
        saved_audio_paths = save_segmented_audio(result['segment_audio_time'], result['audio_path'])

        # Create a ZIP file in memory
        zip_buffer = BytesIO()
        with ZipFile(zip_buffer, 'w') as zip_file:
            for file_path in saved_audio_paths:
                # Add each file to the ZIP archive
                zip_file.write(file_path, os.path.basename(file_path))

        zip_buffer.seek(0)

        # Send the ZIP file as a response
        return send_file(
            zip_buffer,
            mimetype='application/zip',
            as_attachment=True,
            download_name='segmented_audios.zip'
        )
    else:
        return jsonify(result), 400



stopping_list = ["۝"]
mendatory_stopping = ["\u06D8", "۞", "۩"]
non_stopping_list = ["\u06D9"]

# Flask POST route to process Ayat and return segmented text list
@app.route('/quran_segmented', methods=['POST'])
def process_quran_ayat_api():
    data = request.get_json()

    # Get input values from the POST request
    surah_index = data['surah_index']
    start_ayah = data['start_ayah']
    end_ayah = data['end_ayah']
    words_per_segment = data['words_per_segment']

    # Quran CSV file path
    file_path = './quran_updated.csv'
    
    # Load Quran data
    quran_data = load_quran_data(file_path)
    surah_index = int(surah_index)
    start_ayah = int(start_ayah)
    end_ayah = int(end_ayah)
    words_per_segment = int(words_per_segment)
    
    # Get the Ayat within the specified Surah and range
    ayat_data = get_ayat_range(quran_data, surah_index, start_ayah, end_ayah)
    if ayat_data.empty:
        return jsonify({"error": "No Ayat found for the given Surah and Ayat range."}), 404
    
    # Concatenate and segment the Ayat data
    segments = concatenate_and_segment(ayat_data, words_per_segment, stopping_list, non_stopping_list, mendatory_stopping, surah_index, start_ayah, end_ayah)
    
    # Remove empty segments
    segments = [segment for segment in segments if segment.strip()]
    seged = update_segments_list(surah_index
    , start_ayah, end_ayah, segments)
    return jsonify({"segments": seged})

@app.route("/evaluate", methods=["POST"])
def evaluate():
    try:
        correct_ayat = request.form.get("correct_ayat")
        print(correct_ayat)
        if not correct_ayat:
            return jsonify({"error": "Missing correct_ayat parameter"}), 400

        if "audio_file" not in request.files:
            return jsonify({"error": "Missing audio file"}), 400

        audio_file = request.files["audio_file"]
        file_path = "temp_audio.wav"
        audio_file.save(file_path)

        transcription = transcribe_audio(file_path)
        print(transcription)
        transcription_no_diacritics = remove_diacritics(transcription)
        result = calculate_error_rate(correct_ayat, transcription, transcription_no_diacritics)
        print(result)
    
        os.remove(file_path)

        return jsonify(result)

    except Exception as e:
        return jsonify({"error": str(e)}), 500
if __name__ == '__main__':
    app.run(port=3003)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3003
Press CTRL+C to quit


قُلْ أَعُوذُ بِرَبِّ النَّاسِ


c:\WorkingFolder\FYP\Ayat-segmentation\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:668: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.8` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
127.0.0.1 - - [21/Mar/2025 11:32:53] "GET /evaluate HTTP/1.1" 405 -
127.0.0.1 - - [21/Mar/2025 11:33:22] "POST /evaluate HTTP/1.1" 200 -


قُلْ عَأُوزُوا نَاثٌ 
قُلْ أَعُوذُ بِرَبِّ النَّاسِ--------------قُلْ عَأُوزُوا نَاثٌ --------------قل عأوزوا ناث 
['قُلْ', 'أَعُوذُ', 'بِرَبِّ', 'النَّاسِ'] ['قُلْ', 'عَأُوزُوا', 'نَاثٌ'] ['قل', 'عأوزوا', 'ناث']
{'error_rate': 0.75, 'wrong_words': ['أَعُوذُ', 'بِرَبِّ', 'النَّاسِ']}
C:\WorkingFolder\FYP\Ayat-segmentation\000_versebyverse_5\001006.mp3


...

الرَّحْمَـٰنِ الرَّحِيمِ 
ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ--------------الرَّحْمَـٰنِ الرَّحِيمِ --------------الرحمـٰن الرحيم 
['ٱلرَّحۡمَٰنِ', 'ٱلرَّحِيمِ'] ['الرَّحْمَـٰنِ', 'الرَّحِيمِ'] ['الرحمـٰن', 'الرحيم']
{'error_rate': 0.0, 'wrong_words': []}
ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ


127.0.0.1 - - [21/Mar/2025 15:25:47] "POST /evaluate HTTP/1.1" 200 -


الرَّحْمَـٰنِ الرَّحِيمِ 
ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ--------------الرَّحْمَـٰنِ الرَّحِيمِ --------------الرحمـٰن الرحيم 
['ٱلرَّحۡمَٰنِ', 'ٱلرَّحِيمِ'] ['الرَّحْمَـٰنِ', 'الرَّحِيمِ'] ['الرحمـٰن', 'الرحيم']
{'error_rate': 0.0, 'wrong_words': []}
ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ


127.0.0.1 - - [21/Mar/2025 15:27:33] "POST /evaluate HTTP/1.1" 200 -


الرَّحْمَـٰنِ رَحِيمٍ 
ٱلرَّحۡمَٰنِ ٱلرَّحِيمِ--------------الرَّحْمَـٰنِ رَحِيمٍ --------------الرحمـٰن رحيم 
['ٱلرَّحۡمَٰنِ', 'ٱلرَّحِيمِ'] ['الرَّحْمَـٰنِ', 'رَحِيمٍ'] ['الرحمـٰن', 'رحيم']
{'error_rate': 0.0, 'wrong_words': []}
